In [1]:
import random

import pandas as pd

In [54]:
usd_avg = 92.52

In [2]:
poses_start = pd.read_excel('data/brok_taxes_2024.xlsx', sheet_name='open_poses_start')
poses_start = poses_start[poses_start['ticker'] != 'RURCUR']

In [3]:
poses_end = pd.read_excel('data/brok_taxes_2024.xlsx', sheet_name='open_poses_end')
poses_end = poses_end[poses_end['ticker'] != 'RURCUR']

In [6]:
poses_start.sample(5)

,ticker,count_start,price_start,volume_start,count_end,price_end,volume_end,balance
1,PGNY,26,31.15,809.90,26,37.18,966.68,1231.66
5,SPXU,0,0.00,0.00,2198,8.60,18902.80,18902.80
2,SEMR_IPO,1,8.14,8.14,1,13.66,13.66,14.00
0,F.GOGR_USD,42,13.05,548.10,42,12.67,531.97,548.10
6,TZA,0,0.00,0.00,28,19.95,558.60,544.60


In [7]:
trades = pd.read_excel('data/brok_taxes_2024.xlsx', sheet_name='trades_pd')
trades = trades[trades['ticker'] != 'RURCUR']


In [8]:
poses_end.sample(5)

,ticker,count_start,price_start,volume_start,count_end,price_end,volume_end,balance
4,SPBE,NaN,NaN,NaN,301,NaN,NaN,3644.76
2,SEMR_IPO,NaN,NaN,NaN,1,NaN,NaN,14.00
6,TZA,NaN,NaN,NaN,28,NaN,NaN,544.60
5,SPXU,NaN,NaN,NaN,1099,NaN,NaN,9452.40
1,PGNY,NaN,NaN,NaN,26,NaN,NaN,1231.66


In [33]:
trades.sample(5)

,ticker,dir,price,count,total,com_brok,com_ex
7,VXX,Продажа,14.02,-2066,28965.32,24.80,0
0,TMV,Купля,30.89,140,-4324.57,1.68,0
6,VXX,Купля,14.90,584,-8701.13,7.01,0
1,TMV,Продажа,34.31,-44,1509.64,1.20,0
4,VXX,Купля,14.11,142,-2003.34,1.70,0


In [10]:
deriv_tickers = "UVXY TQQQ UPRO URTY KOLD SPXU TZA SOXS BOIL SQQQ TMF TMV VXX NUGT JNUG YINN UNL".split(' ')

In [12]:
share_tickers = list(set(trades['ticker']).union(set(poses_end['ticker'])).union(set(poses_start['ticker'])) - set(deriv_tickers))


In [44]:
exclude_tickers = share_tickers

In [45]:
def filter_by_ticker(df, exclude_tickers):
    return df[~df['ticker'].isin(exclude_tickers)].copy()

In [46]:
def calc_deduct_expenses(poses, trades, exclude_tickers=[]):
    poses = filter_by_ticker(poses, exclude_tickers)
    trades = filter_by_ticker(trades, exclude_tickers)
    iterator = poses[poses['count_end'] != 0].iterrows()
    open_end = 0
    ixs = []
    tickers = []
    balance_sums = []
    trade_counts = []


    while True:
        try: 
            pose = next(iterator)
            purcahse_count = 0
            balance_sum = .0
            balance_price = pose[1]['balance'] / pose[1]['count_end']
            for row in trades[(trades['ticker'] == pose[1]['ticker']) &
                                     (trades['dir'] == 'Купля')].iterrows():

                purcahse_count += row[1]['count']
                balance_sum += row[1]['count'] * balance_price
                if purcahse_count > pose[1]['count_end']:
                    balance_sum -= (purcahse_count - pose[1]['count_end']) * balance_price
                    purcahse_count = pose[1]['count_end']
                if pose[1]['count_end'] == purcahse_count:
                    break
            tickers.append(pose[1]['ticker'])
            balance_sums.append(balance_sum)
            trade_counts.append(purcahse_count)

            open_end += 1
            ixs.append(pose[0])
        except StopIteration:
            break
    return pd.DataFrame(data={'ticker': tickers, 'balance': balance_sums, 'trade_count': trade_counts})

In [47]:
deduct_expenses = calc_deduct_expenses(poses_end, trades[::-1], exclude_tickers)

In [48]:
deduct_expenses

,ticker,balance,trade_count
0,SOXS,0.00,0
1,SPXU,0.00,0
2,TZA,0.00,0
3,VXX,1202.19,77


In [49]:
def calc_deduct_profits(poses, trades, exclude_tickers=[]):
    poses = filter_by_ticker(poses, exclude_tickers)
    trades = filter_by_ticker(trades, exclude_tickers)
    iterator = poses[poses['count_end'] != 0].iterrows()
    open_end = 0
    ixs = []
    tickers = []
    balance_sums = []
    trade_counts = []

    while True:
        try: 
            pose = next(iterator)
            sell_count = 0
            balance_sum = .0
            balance_price = pose[1]['balance'] / pose[1]['count_end']
            for row in trades[(trades['ticker'] == pose[1]['ticker']) &
                                     (trades['dir'] == 'Продажа')].iterrows():

                sell_count += -row[1]['count']
                balance_sum += -row[1]['count'] * balance_price
                if sell_count > pose[1]['count_end']:
                    balance_sum -= (sell_count - pose[1]['count_end']) * balance_price
                    sell_count = pose[1]['count_end']
                if pose[1]['count_end'] == sell_count:
                    break
            tickers.append(pose[1]['ticker'])
            balance_sums.append(balance_sum)
            trade_counts.append(sell_count)

            open_end += 1
            ixs.append(pose[0])
        except StopIteration:
            break
    return pd.DataFrame(data={'ticker': tickers, 'balance': balance_sums, 'trade_count': trade_counts})

In [50]:
deduct_profits = calc_deduct_profits(poses_start, trades, exclude_tickers)

In [51]:
deduct_profits

,ticker,balance,trade_count
0,SOXS,0.00,0
1,SPXU,9451.40,1099
2,TZA,0.00,0
3,VXX,20046.85,1284


In [52]:
def calc_profit_loss(trades, poses_current, poses_prev, exclude_tickers=[]):
    trades = filter_by_ticker(trades, exclude_tickers)
    reverse_trades = trades[::-1]
    deduct_profits = calc_deduct_profits(poses_prev, trades, exclude_tickers)
    deduct_expenses = calc_deduct_expenses(poses_current, reverse_trades, exclude_tickers)
    sales_only_mask = trades['dir'] == 'Продажа'
    purchases_only_mask = trades['dir'] == 'Купля'
    profit = trades[sales_only_mask]['total'].sum()
    expense = (-trades[purchases_only_mask]['total'].sum()) + \
                trades['com_brok'].sum() + \
                trades['com_ex'].sum() + \
                deduct_profits['balance'].sum() - \
                deduct_expenses['balance'].sum()
    return pd.DataFrame({'profit': [profit], 'expense': [expense], 'result': [profit - expense]})
            

In [53]:
calc_profit_loss(trades, poses_end, poses_start, exclude_tickers)

,profit,expense,result
0,42540.22,45378.47,-2838.25


In [29]:
div = 250.15 - 37.52 # deriv only
div

212.63

In [57]:
42540.22	+ div

42752.85

In [23]:
interest_margin_total = 434.94

In [56]:
(45378.47 + interest_margin_total) * usd_avg

4238656.693200001